In [598]:
# This class will read all txt files within a folder
# Input: Number of files, directory of folder
# Intermediate Step: Single matrix with all datasets
# Output: A matrix with -> Class  Class  Class  ...
#                          Class  0      x2      ...
#                          Class  x1     0      ...
#                           ...   ...    ...    ...
# where x1, x2, ... represent a sum of all feature deviations from their respective means 

# File Requirements:
# Files will need to be named "emgset*.txt"
# Files will need to be already classified
# Directory input must be of Dataset/
# Data files come in array of [Time, 8*7 features, Classification]
# Have it ordered: Gesture 1 datasets, then Gesture 2 datasets, .... 


class stdDevMatrix:
    def __init__(self, N, Dir):
        self.N = N
        self.Dir = Dir
    
    def createMatrix(self):
        fname = self.Dir+"features_emgset" 
        
        newArray = [] 
        
        # N number of files will be evaluated 
        # First we have to create a singular matrix with all the data
        for x in range(0, self.N):
            fdirect = fname + str(x)
            with open (fdirect, "r") as f:
                data = []
                data = f.read()
                data = data.split("\n")
            
            for y in range(0,len(data)):
                myList = []
                myList = data[y].split(",")
                # myList= data[y][len(data[y])-1].replace('.0', '')
                # myList.replace(';','')
                # myList = [int(x) for x in myList.split(';')]
                for x in range(len(myList)-1):
                    myList[x] = float(myList[x])
                    
                myList = myList[0:len(myList)-1]
                newArray.append(myList)
                # convert the data set from a list of strings to a list of ints 

        newArray = newArray[0:len(newArray)-1]
        
        # CHECKPOINT 2
        
        sumArray = [] 
        
        # [Class, sum of 8x7 features, totalSamples] = Output
        # newArray right now = [time, 8*7 features, class]
        for y in range(0, len(newArray)):

            # the last value of the list in the list = class
            z = (len(newArray[y])-1)
            
            # when the summation array has elements
            if (len(sumArray) != 0 and z > 0):
                
                # parse through the sumArray in order to access each class managed so far 
                for j in range(0, len(sumArray)):
                    
                    # find existing class in array
                    if (newArray[y][z] == sumArray[j][0]):
                        
                        # holds the dimension of the size of the list in sumArray
                        dimNew = len(sumArray[j])

                        # now add up the current values within the newArray and new sample
                        for alpha in range (1, dimNew-1):
                            sumArray[j][alpha] = sumArray[j][alpha] + newArray[y][alpha]

                        # add sample count (located at the last index)
                        sumArray[j][dimNew-1] +=  1
                        break
                    
                else:
                    # takes the class from the last element within the array 
                    myList = []
                    myList = newArray[y]

                    # grabs the last element which is the class
                    newClass = myList[len(myList)-1]

                    newData = []
                    # grabs the data between first and last element which is all the features
                    newData = newArray[y][1:(len(myList)-1)]

                    # this is the first instance of the sample
                    newData.append(1)
                    newData.insert(0, newClass)

                    # adds the new set of data into the sumArray
                    sumArray.append(newData)

            # this is the case that an existing class was not found in the loop
            elif (len(sumArray) == 0 and z > 0):
                # takes the class from the last element within the array 
                myList = []
                myList = newArray[y]
                # grabs the last element which is the class
                newClass = myList[len(myList)-1]

                newData = []
                # grabs the data between first and last element which is all the features
                newData = newArray[y][1:(len(myList)-1)]

                # this is the first instance of the sample
                newData.append(1)
                newData.insert(0, newClass)

                # adds the new set of data into the sumArray
                sumArray.append(newData)

        # CHECKPOINT 3

        # Now we have [[Class, sum of 72 features, # of samples], [Class, sum of 72 features, # of samples], ...]
        # print("\nSummation Matrix")
        # print(sumArray)
        
        size = len(sumArray[0]) - 2
        total = [0]*size
        # total = [sum of every channel's feature across all classes]
        
        # featureTotal = the total number of samples across all classes
        featureTotal = 0
        
        # loops through the number of different classes and their sums within the sum array to find average 
        for g in range (len(sumArray)):

            # divide the sums of the 72 features by the number of samples
            indexSamples = len(sumArray[g])-1   
            featureTotal = featureTotal+ sumArray[g][indexSamples]

            for h in range(1, indexSamples):
                
                # add all the totals will be used later 
                total[h-1] = total[h-1] + sumArray[g][h]
                
                # divides the sums of each by the number of samples to find average 
                sumArray[g][h] = sumArray[g][h]/sumArray[g][indexSamples]

        # Now we have a matrix with the average of each of its features
        # Class1 and Class 2 Deviation = sqrt((C1F1+C2F1)^2 + (C1F2 + C2F2)^2 + ...)
        # print("\nAverage Matrix")
        # print(sumArray)
        
        finalMatrix = []
        
        for x in range(len(sumArray)):
            for y in range(len(sumArray)):
                stdSum = 0
                if (x != y):
                    for c in range(1, len(sumArray[x])-1):
                        # loop through the sumArray columns 
                        stdSum = stdSum + (sumArray[x][c]-sumArray[y][c])**(2)
                    # Now square root the summation
                    finalMatrix.append(stdSum**(0.5))
                else:
                    finalMatrix.append(0)
                    
        print("\nFinal Matrix")
        for x in range(len(sumArray)):
            print(finalMatrix[0+x*len(sumArray):len(sumArray)+len(sumArray)*x])

In [603]:
checkTest = stdDevMatrix(75, "Dataset/")
checkTest.createMatrix()


Final Matrix
[0, 144.95933672066877, 122.97056592716872, 93.20918538295076, 98.16493372848348]
[144.95933672066877, 0, 129.78724160760947, 117.74096877502467, 148.72533016092927]
[122.97056592716872, 129.78724160760947, 0, 116.43800858190372, 72.42536463472932]
[93.20918538295076, 117.74096877502467, 116.43800858190372, 0, 71.70041174111532]
[98.16493372848348, 148.72533016092927, 72.42536463472932, 71.70041174111532, 0]
